# Classification (분류)
___
## Naive Bayes (나이브 베이지안 분류기)
✔️ 조건부 확률 기반 방법 <br>
✔️ (예) 날씨에 따라 필드에 나갈 지 말지를 결정하려고 하는 문제 

#### 조건부 확률
- **교집합** : <u>사건 A와 B가 동시에</u> 일어날 확률  <br>
$$ P(A\cap B) = P(A) * P(B|A)  = P(B) * P(A|B)$$

#### 베이지안 룰 
$$ P(A|B) = \frac{P(A) * P(B|A)}{P(B)}$$

- P(A) : 이미 일어난 사건의 확률 (= 사전확률, prior probability)
- P(B|A) : 이미 일어난 상황 A에서 B가 일어날 확률 (= 가능도, likelihood)
- P(B) : 모든 가설에 대한 증거가 발생할 확률 (= 에비던스 모델, evidence model)
- P(A|B) : B가 일어났을 때, A가 일어날 확률 (= 사후 확률, posterior probability)



In [1]:
from google.colab import files
uploaded= files.upload()
filename= list(uploaded.keys())[0]
filename

Saving weather.csv to weather.csv


'weather.csv'

* 자료 읽기

In [2]:
import pandas as pd 
data =pd.read_csv(filename)
data

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


* Navie Bayes 모듀 중 MultinomialNB() 가져와 인스턴스화 하기

In [3]:
from sklearn.naive_bayes import MultinomialNB
multinomial_model = MultinomialNB()
multinomial_model

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

* 데이터 전처리
   - Category 문자 데이터를 숫자 카테고리로 변환 (Map 활용)


In [8]:
# mapping dictionary 만들기 
outlook_dic = {'overcast':0, 'rainy':1, 'sunny':2}
temperature_dic = {'cool':0, 'hot':1, 'mild':2}
humidity_dic = {'high':0, 'normal':1}
windy_dic = {False:0, True:1}

In [5]:
# 딕셔너리를 활용해 데이터 맵핑 
data['outlook'] = data['outlook'].map(outlook_dic)
data['temperature'] = data['temperature'].map(temperature_dic)
data['humidity'] = data['humidity'].map(humidity_dic)
data['windy'] = data['windy'].map(windy_dic)

In [6]:
# 변형된 데이터 확인
data 

,outlook,temperature,humidity,windy,play
0,0,1,0,0,yes
1,0,0,1,1,yes
2,0,2,0,1,yes
3,0,1,1,0,yes
4,1,2,0,0,yes
5,1,0,1,0,yes
6,1,0,1,1,no
7,1,2,1,0,yes
8,1,2,0,1,no
9,2,1,0,0,no


* Multinomial 모델 학습

In [9]:
multinomial_model.fit(data.iloc[:, :4], data['play'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
# 예측 
multinomial_model.predict([[1,2,0,1]])

array(['yes'], dtype='<U3')

In [11]:
# 계산된 확률 
multinomial_model.predict_proba([[2,2,0,1]])

array([[0.57311795, 0.42688205]])

In [13]:
# 더 높은 확율을 가지는 것을 정답으로 분류 
multinomial_model.predict([[2,2,0,1]])

array(['no'], dtype='<U3')

### 연속적인 데이터(iris)에 대한 나이브 베이지안 분류기

* 데이터 준비

In [14]:
from sklearn.datasets import load_iris
iris = load_iris()

In [16]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


* 가우시안 모델 인스턴스화

In [17]:
from sklearn.naive_bayes import GaussianNB
gaussian_model = GaussianNB()

* 데이터를 학습데이터와 테스트 데이터로 분할

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_df.iloc[:, :4], 
                                                    iris_df['species'], 
                                                    test_size = 0.33)

In [20]:
# 학습
gaussian_model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

* 테스트 성능평가

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, gaussian_model.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.92      0.92      0.92        12
           2       0.96      0.96      0.96        24

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50



In [22]:
print(confusion_matrix(y_test, gaussian_model.predict(X_test)))

[[14  0  0]
 [ 0 11  1]
 [ 0  1 23]]
